In [3]:
import numpy as np
from BF_checkers import *
from BF_generators import *
from BF_properties import *

In [4]:
def IntsToBitsFast_lr (ints, size):
    '''
    return an array of bits of length 'size' for the set of integers ints such that the MSB is at column 0
    source: https://stackoverflow.com/questions/
    ints: numpy array of integers
    size: required size of the array of bits
    '''
    return (((ints[:,None] & (1 << np.arange(size, dtype = 'int64'))[::-1])) > 0).astype(int)

In [5]:
def make_EF_from_IEF(k, f, inp_list):
    
    ineff_ind = []
    z = bf(k, f).indices()[0]
    o = bf(k, f).indices()[1]
    for i in z:
        z_ele = list(itemgetter(*z[i])(f))
        o_ele = list(itemgetter(*o[i])(f))
        if z_ele == o_ele:
            ineff_ind.append(i)
    if ineff_ind == []:
        return inp_list, f
    else:
        union_result = set.union(*(set(z[k]) for k in ineff_ind))
        EF_f = "".join(c for i, c in enumerate(f) if i not in union_result)
        ineff_ind_usable = sorted(k-np.array(ineff_ind))
        EF_inp_list = [inp_list[i] for i in (range(k)) if i not in ineff_ind_usable]
        return EF_inp_list, EF_f

# IMR to BF

We want to map an IMR to the corresponding BF.
The IMR is defined by
$$S_i(t+1) = \begin{cases}
+1 ~~~if \sum_j J_{ij}S_j > 0\\
-1 ~~~if \sum_j J_{ij}S_j < 0\\
S_i(t) ~~~if \sum_j J_{ij}S_j = 0
\end{cases}
$$
Here, $J_{ij}$ indicates the sign of the edge from node $j$ to node $i$ and hence can be either +1 or -1. In this formalism the state of any node can be either +1 or -1. If all regulators are external then we can map a $k$-input IMR to a $k$-input BF if $k$ is odd otherwise it will map to a $k+1$-input, where the added implicit input is $S_i(t)$.<br> 
First, for a given sign combination we calculate the values of $\sum_j J_{ij}S_j$. Here, first we have mapped the {0,1} to {-1, 1} with the transformation $S'_{i} = (2S_i-1)$ that maps 0 to -1 and 1 to 1. So, we can compute the quantity $\sum_j J_{ij}S_j$ for each input combination now. Once, this computation is done, we replace the place where $\sum_j J_{ij}S_j = 0$ by the respective entries from the column associated with $S_i(t)$. Then, we apply sign function on it. Finally, we change the -1's to 0 and keep the 1's as it is.Once we join the array, we obtain the equivalent Boolean string. If $k$ is odd then the quantity $\sum_j J_{ij}S_j$ will never be equal to 0 then we directly proceed with the application of sign function. This function outputs a string length $2^k$ if $k$ is odd and $2^{k+1}$ if $k$ is even. Each bit in this string corresponds to the output corresponding to the respective entry when written in canonical order with correspondence of '0' to '-1' and '1' to '1'. <br>

When there is a self-loop we map it to a k-input BF (which may or may not be EF).

(a) When all regulators $k$ regulators are external. If k is odd and even if there is a self-regulation,
    this code is still applicable.

In [6]:
def IMR_to_EF_all_external_regulators(k, sign, self_loop_index):
    '''
    This code is suitable when all regulators are external. If k is odd and even if there is a self-regulation,
    the same code is applicable.
    
    Input
    -----
    k: Number of regulators
    sign: k-length array of +1 and -1 which corresponds to the signs of the regulators
    self_loop_index: The index where the self-loop will be added. Only relevant if k is even.
    
    Output
    -----
    A binary string of length 2^k (if k odd) or 2^(k+1) (if k even)
    '''
    if k%2 == 0:
        all_inp = IntsToBitsFast_lr (np.array([i for i in range(2**(k+1))]), k+1)
        inps_excl_self_loop = np.delete(all_inp, self_loop_index, axis=1)
        j_ij_s_j = np.sum(sign * (2 * inps_excl_self_loop - 1), axis=1)
        self_inp_col = all_inp[:, self_loop_index]
        j_ij_s_j_0_replaced = np.where(j_ij_s_j == 0, (2 * self_inp_col - 1), j_ij_s_j)
        sign_result = np.sign(j_ij_s_j_0_replaced)
        modified_array = np.where(sign_result == -1, 0, sign_result)
        EF_logic_func_string = ''.join(modified_array.astype(str))
        
    else:
        all_inp = IntsToBitsFast_lr (np.array([i for i in range(2**(k))]), k)
        j_ij_s_j = np.sum(sign * (2 * all_inp[:, :k] - 1), axis=1)
        sign_result = np.sign(j_ij_s_j)
        modified_array = np.where(sign_result == -1, 0, sign_result)
        EF_logic_func_string = ''.join(modified_array.astype(str))
        
    return EF_logic_func_string

In [7]:
IMR_to_EF_all_external_regulators(4, [-1, 1, 1, -1], 0)

'00101011000000101011111100101011'

(b) This is applicable if k is even and there is a self-regulation.

In [8]:
def IMR_to_EF_with_self_regulation_with_even_k(k, sign, self_index):
    '''
    Use this code to obtain the EF which corresponds to an IMR with k regulators including a self-regulation
    and k is even.
    Input
    -----
    k: Number of regulators
    sign: k-length array of +1 and -1 which corresponds to the signs of the regulators
    self_index: The index of the self-regulator within the inputs.
    
    Output
    -----
    A binary string of length 2^k or 2^(k-1)
    '''
    if k%2 == 0:
        all_inp = IntsToBitsFast_lr (np.array([i for i in range(2**k)]), k)
        j_ij_s_j = np.sum(sign * (2 * all_inp[:, :k] - 1), axis=1)
        last_inp_col = all_inp[:, self_index]
        j_ij_s_j_0_replaced = np.where(j_ij_s_j == 0, (2 * last_inp_col - 1), j_ij_s_j)
        sign_result = np.sign(j_ij_s_j_0_replaced)
        modified_array = np.where(sign_result == -1, 0, sign_result)
        logic_func_string = ''.join(modified_array.astype(str))
        EF_logic_string = make_EF_from_IEF(k, logic_func_string, [i for i in range (k)])[1]
    return EF_logic_string

In [9]:
IMR_to_EF_with_self_regulation_with_even_k(4, [-1,-1, 1, 1], 1)

'01110001'

# BMR to BF

BMR is defined as the same way as IMR. The only difference is, here the states of the nodes are 0 or 1.

In [10]:
def BMR_to_EF_all_external_regulators(k, sign, self_loop_index):
    '''
    This code is suitable when all regulators are external.
    
    Input
    -----
    k: Number of regulators
    sign: k-length array of +1 and -1 which corresponds to the signs of the regulators
    self_loop_index: The index where the self-loop will be added. Only relevant if k is even.
    
    Output
    -----
    A binary string of length 2^(k+1)
    '''
    all_inp = IntsToBitsFast_lr (np.array([i for i in range(2**(k+1))]), k+1)
    inps_excl_self_loop = np.delete(all_inp, self_loop_index, axis=1)
    j_ij_s_j = np.sum(sign * inps_excl_self_loop, axis=1)
    self_inp_col = all_inp[:, self_loop_index]
    j_ij_s_j_0_replaced = np.where(j_ij_s_j == 0, self_inp_col, j_ij_s_j)
    sign_result = np.sign(j_ij_s_j_0_replaced)
    modified_array = np.where(sign_result == -1, 0, sign_result)
    EF_logic_func_string = ''.join(modified_array.astype(str))
    return EF_logic_func_string

In [11]:
BMR_to_EF_all_external_regulators(3, [1,-1, 1], 1)

'0100110111011111'

In [14]:
def BMR_to_EF_with_self_regulation(k, sign, self_index):
    '''
    Use this code to obtain the EF which corresponds to an BMR with k regulators including a self-regulation
    
    Input
    -----
    k: Number of regulators
    sign: k-length array of +1 and -1 which corresponds to the signs of the regulators
    self_index: The index of the self-regulator within the inputs.
    
    Output
    -----
    A binary string of length 2^k or 2^(k-1)
    '''
    all_inp = IntsToBitsFast_lr (np.array([i for i in range(2**k)]), k)
    j_ij_s_j = np.sum(sign * all_inp[:, :k], axis=1)
    last_inp_col = all_inp[:, self_index]
    j_ij_s_j_0_replaced = np.where(j_ij_s_j == 0, last_inp_col, j_ij_s_j)
    sign_result = np.sign(j_ij_s_j_0_replaced)
    modified_array = np.where(sign_result == -1, 0, sign_result)
    logic_func_string = ''.join(modified_array.astype(str))
    EF_logic_string = make_EF_from_IEF(k, logic_func_string, [i for i in range (k)])[1]
    return EF_logic_string

In [15]:
BMR_to_EF_with_self_regulation(3, [1,1,-1], 1)

'00111011'